## RAGRAS를 사용한 평가

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

### RAGAS (Retrieval-Augmented Generation Assessment Suite)
RAGAS는 RAG(Retrieval-Augmented Generation) 시스템의 응답 품질을 자동으로 평가하기 위한 프레임워크이다.

In [2]:
# ! pip install ragas evaluate datasets

In [3]:
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

import ragas
print(ragas.__version__)

0.3.7


In [4]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    ContextPrecision,
    ContextRecall,
    Faithfulness,
    ResponseRelevancy
)

# 데이터 준비
data = {
    "question": ["딥러닝이란 무엇인가?"],
    "contexts": [["딥러닝은 인공신경망을 사용하는 머신러닝의 한 분야이다."]],
    "answer": ["딥러닝은 머신러닝의 하위 분야로 신경망을 사용한다."],
    "ground_truth": ["딥러닝은 인공신경망을 기반으로 한 머신러닝 기술이다."]
}
dataset = Dataset.from_dict(data)

# 메트릭 객체 인스턴스 생성
metrics = [
    Faithfulness(),
    ResponseRelevancy(),
    ContextPrecision(),
    ContextRecall()
]

# 평가 실행
results = evaluate(dataset=dataset, metrics=metrics)
print(results)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


{'faithfulness': 1.0000, 'answer_relevancy': 0.8375, 'context_precision': 1.0000, 'context_recall': 1.0000}


### RAG + RAGAS 평가 자동화 에이전트 구현

In [5]:
# ==============================================================
# 복합 RAG + RAGAS 평가 자동화 에이전트 예제 (LangChain v1.0 + RAGAS 0.3.7)
# ==============================================================

# 설치 필요:
# pip install langchain langchain-openai langchain-chroma ragas evaluate datasets pandas

import os
from dotenv import load_dotenv
import pandas as pd
from typing import List, Dict

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.agents import create_agent
from langchain.tools import tool

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    ContextPrecision,
    ContextRecall
)

# --------------------------------------------------------------
# 1. 환경 설정
# --------------------------------------------------------------
# load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# --------------------------------------------------------------
# 2. 예시 문서 데이터
# --------------------------------------------------------------
docs = [
    Document(page_content="인공지능은 인간의 지능을 모방하는 기술이며, 다양한 분야에서 활용된다."),
    Document(page_content="머신러닝은 데이터를 이용해 학습하는 AI의 한 영역이다."),
    Document(page_content="딥러닝은 인공신경망 기반의 머신러닝 기법으로 이미지, 음성 인식에 뛰어나다."),
    Document(page_content="자연어처리는 인간의 언어를 이해하고 생성하는 AI 기술이다."),
    Document(page_content="강화학습은 보상 기반으로 학습하여 행동을 최적화하는 AI 분야이다.")
]

# --------------------------------------------------------------
# 3. 벡터스토어 생성
# --------------------------------------------------------------
db = Chroma.from_documents(docs, embedding)

# --------------------------------------------------------------
# 4. 검색 Tool 정의
# --------------------------------------------------------------
@tool
def semantic_retrieve(query: str) -> str:
    """주어진 질문과 관련된 문서를 검색하여 반환한다."""
    retriever = db.as_retriever(search_kwargs={"k": 3})
    results = retriever.invoke(query)
    return "\n".join([r.page_content for r in results])

# --------------------------------------------------------------
# 5. 에이전트 생성
# --------------------------------------------------------------
rag_agent = create_agent(
    model=llm,
    tools=[semantic_retrieve],
    system_prompt=(
        "당신은 검색과 문서 요약을 수행하는 RAG 분석 에이전트이다. "
        "주어진 질문에 대해 관련 문서를 검색하고, 근거를 바탕으로 신뢰성 있는 답변을 생성하라. "
        "모르면 추측하지 말고 '관련 정보가 없습니다.'라고 답하라."
    )
)

# --------------------------------------------------------------
# 6. 다중 질의 실행 및 결과 저장
# --------------------------------------------------------------
questions = [
    "딥러닝이란 무엇인가?",
    "자연어처리는 어떤 기술인가?",
    "강화학습은 어떤 원리로 동작하는가?",
    "머신러닝과 딥러닝의 차이점을 설명해줘.",
    "AI는 인간의 사고를 완전히 대체할 수 있는가?"
]

answers, contexts = [], []

for q in questions:
    result = rag_agent.invoke({"messages": [{"role": "user", "content": q}]})
    answer = result["messages"][-1].content
    print(f"[질문] {q}\n[답변] {answer}\n{'-'*60}")
    answers.append(answer)
    contexts.append([doc.page_content for doc in docs])

# --------------------------------------------------------------
# 7. 평가용 데이터셋 구성
# --------------------------------------------------------------
data = {
    "question": questions,
    "contexts": contexts,
    "answer": answers,
    "ground_truth": [
        "딥러닝은 인공신경망을 사용하는 머신러닝의 하위 분야이다.",
        "자연어처리는 인간의 언어를 이해하고 생성하는 인공지능 기술이다.",
        "강화학습은 보상 기반으로 학습하여 최적의 행동을 선택하는 방법이다.",
        "딥러닝은 머신러닝의 일부로, 신경망을 사용한다.",
        "AI는 인간 사고를 완전히 대체하기 어렵다."
    ]
}

dataset = Dataset.from_dict(data)

# --------------------------------------------------------------
# 8. RAGAS 평가 수행
# --------------------------------------------------------------
metrics = [
    Faithfulness(),
    ResponseRelevancy(),
    ContextPrecision(),
    ContextRecall()
]

print("\n[🔍 RAGAS 평가 시작]")
results = evaluate(dataset=dataset, metrics=metrics)

# --------------------------------------------------------------
# 9. 평가 결과 출력
# --------------------------------------------------------------
print("\n=== RAGAS 평가 결과 ===")
print(results)

# DataFrame 형태로 보기 좋게 표시
df_results = pd.DataFrame([results])
print("\n평가 점수표:\n", df_results)


[질문] 딥러닝이란 무엇인가?
[답변] 딥러닝은 인공신경망 기반의 머신러닝 기법으로, 주로 이미지 및 음성 인식 분야에서 뛰어난 성능을 발휘합니다. 이는 머신러닝의 한 영역으로, 데이터를 이용해 학습하는 방식입니다.
------------------------------------------------------------
[질문] 자연어처리는 어떤 기술인가?
[답변] 자연어처리(Natural Language Processing, NLP)는 인간의 언어를 이해하고 생성하는 인공지능 기술입니다. 이 기술은 컴퓨터가 자연어를 처리하고 분석하여 의미를 이해할 수 있도록 돕습니다. 자연어처리는 텍스트 분석, 언어 번역, 감정 분석 등 다양한 분야에서 활용됩니다.
------------------------------------------------------------
[질문] 강화학습은 어떤 원리로 동작하는가?
[답변] 강화학습은 보상 기반으로 학습하여 행동을 최적화하는 AI 분야입니다. 에이전트는 환경과 상호작용하며, 특정 행동을 취했을 때 받는 보상을 통해 학습합니다. 이 과정을 통해 에이전트는 장기적인 보상을 극대화하는 방향으로 행동을 조정하게 됩니다.
------------------------------------------------------------
[질문] 머신러닝과 딥러닝의 차이점을 설명해줘.
[답변] 머신러닝과 딥러닝의 차이점은 다음과 같습니다:

1. **정의**:
   - **머신러닝**: 데이터를 이용해 학습하는 인공지능(AI)의 한 영역으로, 다양한 알고리즘을 통해 패턴을 인식하고 예측을 수행합니다.
   - **딥러닝**: 머신러닝의 한 하위 분야로, 인공신경망을 기반으로 하여 복잡한 데이터(예: 이미지, 음성)를 처리하는 데 뛰어난 성능을 보입니다.

2. **구조**:
   - 머신러닝은 다양한 알고리즘(예: 결정 트리, 서포트 벡터 머신 등)을 사용하여 데이터를 분석합니다.
   - 딥러닝은 여러 층의 신경망을 통해 데이터를 처리

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.



=== RAGAS 평가 결과 ===
{'faithfulness': 0.6333, 'answer_relevancy': 0.6485, 'context_precision': 0.2233, 'context_recall': 0.8000}

평가 점수표:
                                               scores  \
0  [{'faithfulness': 0.75, 'answer_relevancy': 0....   

                                             dataset binary_columns cost_cb  \
0  {'samples': [user_input='딥러닝이란 무엇인가?' retrieve...             []    None   

                                              traces  \
0  [{'scores': {'faithfulness': 0.75, 'answer_rel...   

                                        ragas_traces run_id  
0  {'bc9c8672-5dcc-4e11-8a58-24a27e6688b4': run_i...   None  


In [6]:
df_results

,scores,dataset,binary_columns,cost_cb,traces,ragas_traces,run_id
0,"[{'faithfulness': 0.75, 'answer_relevancy': 0....",{'samples': [user_input='딥러닝이란 무엇인가?' retrieve...,[],None,"[{'scores': {'faithfulness': 0.75, 'answer_rel...",{'bc9c8672-5dcc-4e11-8a58-24a27e6688b4': run_i...,None


In [7]:
# 질문별 상세 평가 결과
# 평가 결과를 DataFrame으로 변환
df = results.to_pandas()
df

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,딥러닝이란 무엇인가?,"[인공지능은 인간의 지능을 모방하는 기술이며, 다양한 분야에서 활용된다., 머신러닝...","딥러닝은 인공신경망 기반의 머신러닝 기법으로, 주로 이미지 및 음성 인식 분야에서 ...",딥러닝은 인공신경망을 사용하는 머신러닝의 하위 분야이다.,0.750000,0.821128,0.333333,1.0
1,자연어처리는 어떤 기술인가?,"[인공지능은 인간의 지능을 모방하는 기술이며, 다양한 분야에서 활용된다., 머신러닝...","자연어처리(Natural Language Processing, NLP)는 인간의 언...",자연어처리는 인간의 언어를 이해하고 생성하는 인공지능 기술이다.,0.666667,0.803317,0.250000,1.0
2,강화학습은 어떤 원리로 동작하는가?,"[인공지능은 인간의 지능을 모방하는 기술이며, 다양한 분야에서 활용된다., 머신러닝...",강화학습은 보상 기반으로 학습하여 행동을 최적화하는 AI 분야입니다. 에이전트는 환...,강화학습은 보상 기반으로 학습하여 최적의 행동을 선택하는 방법이다.,0.250000,0.774015,0.200000,1.0
3,머신러닝과 딥러닝의 차이점을 설명해줘.,"[인공지능은 인간의 지능을 모방하는 기술이며, 다양한 분야에서 활용된다., 머신러닝...",머신러닝과 딥러닝의 차이점은 다음과 같습니다:\n\n1. **정의**:\n - ...,"딥러닝은 머신러닝의 일부로, 신경망을 사용한다.",0.500000,0.844088,0.333333,1.0
4,AI는 인간의 사고를 완전히 대체할 수 있는가?,"[인공지능은 인간의 지능을 모방하는 기술이며, 다양한 분야에서 활용된다., 머신러닝...",현재 인공지능(AI)은 인간의 사고를 모방하고 특정 작업을 수행하는 데 매우 유용하...,AI는 인간 사고를 완전히 대체하기 어렵다.,1.000000,0.000000,0.000000,0.0
